In [ ]:
import sunpy.map

import matplotlib.pyplot as plt
import numpy as np

from sdoml import SDOMLDataset
from timeit import default_timer as timer

First, we will instantiate the ``SDOMLDataset`` class, to load one month of 
the six optically-thin SDO/AIA channels (94A/131A/171A/193A/211A/335A) 
from ``fdl-sdoml-v2/sdomlv2_small.zarr``

In [ ]:
# load the dataset with the six optically thin SDO/AIA channels (94A/131A/171A/193A/211A/335A) 
# from ``fdl-sdoml-v2/sdomlv2_small.zarr``

sdomlds = SDOMLDataset(
    storage_location="gcs",
    zarr_root="fdl-sdoml-v2/sdomlv2_small.zarr/",
    cache_max_size=1*512*512*8192,
    years=["2010"],
    instruments=["AIA"],
    channels=["94A", "131A", "171A", "193A", "211A", "335A"],
    required_keys=None,
)

Next, we will directly access the dataloader using ``__getitem__()``. 

The first data access is relatively slow, as it is retrieved from remote store on Google Cloud Storage; the second data access is faster, as this uses cache.

For more information see, https://zarr.readthedocs.io/en/stable/api/storage.html#zarr.storage.LRUStoreCache

In [ ]:
index = 0
images, metad = sdomlds.__getitem__(index)

In [ ]:
# print the shapes
print(f"images.shape: {images.shape}")
print(f"There are {len(metad.keys)} keys")

### Plotting one set of data

For the one set of images returned from the dataloader, the following for loop creates a set of ``sunpy.map``, which are plotted with the appropriate colorscale.

In [ ]:
plt.figure(figsize=(20, 12))

for img_index in range(0,6):

    # Create a sunpy map with the data
    selected_image = images[0,img_index,:,:]
    selected_headr = {keys: values[img_index] for keys, values in metad.items()}
    my_map = sunpy.map.Map((np.array(selected_image), selected_headr))
    
    # set the index and plot the sunpy.map
    ax = plt.subplot(2, 3, img_index+1, projection=my_map)
    my_map.plot()